# Registering multiple frames to first frame

## Imports and magics

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import ot
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

from otimage import imagerep, readers
from otimage.utils import plot_maxproj

## Implement OT registration (polynomial)

In [3]:
def _compute_ot(pts_1, pts_2, wts_1, wts_2):
    """Normalize weights and compute OT matrix."""
    
    # Normalize weights
    p_1 = wts_1 / np.sum(wts_1)
    p_2 = wts_2 / np.sum(wts_2)

    # Normalized distance matrix 
    M_nn = ot.dist(pts_1, pts_2, metric='sqeuclidean')
    M = M_nn / np.median(M_nn)
    
    # Compute transport plan
    return ot.emd(p_1, p_2, M, log=True)
    

def _ot_reg(pts_1, pts_2, p_mtx):
    """Compute weighted polynomial regression using OT plan"""
    
    # Get pairs of points with values above threshold, and corresponding weights from P matrix
    idx_1, idx_2 = np.nonzero(p_mtx)
    x = pts_1[idx_1]
    y = pts_2[idx_2]
    smp_wt = p_mtx[idx_1, idx_2]

    # Use sklearn to minimize cost function
    model = Pipeline([
        ('poly', PolynomialFeatures(degree=3)),
        ('linear', LinearRegression(fit_intercept=True))
    ])
    model.fit(x, y, linear__sample_weight=smp_wt)

   
    return model


def ot_reg_poly(pts_s, pts_t, wts_s, wts_t, n_iter, pts_0=None):
    """EM-based OT registration method with polynomial model."""
    
    pts = [None] * (n_iter + 1)
    model = [None] * n_iter
    p_mtx = [None] * n_iter
    ot_log = [None] * n_iter
    
    if pts_0 is None:
        pts[0] = pts_s
    else:
        pts[0] = pts_0
    
    for i in range(n_iter):
        
        # E-step: Compute OT between current points and target points
        p_mtx[i], ot_log[i] = _compute_ot(pts[i], pts_t, wts_s, wts_t)
        
        # M-step: Compute new mapping using transport plan
        model[i] =  _ot_reg(pts_s, pts_t, p_mtx[i])
       
        # Update points 
        pts[i + 1] = model[i].predict(pts_s)
       
    debug = {
        'model': model,
        'pts': pts,
        'p_mtx': p_mtx,
        'ot_log': ot_log,
    }
    
    return model[-1], debug

## Load data

In [ ]:
# Frames to load
t_start = 3
n_frames = 20
t_vals = range(t_start, t_start + n_frames)

# Load frames from dataset
img_path = '/home/mn2822/Desktop/WormOT/data/zimmer/raw/mCherry_v00065-00115.hdf5'
with readers.ZimmerReader(img_path) as reader:
    img = [reader.get_frame(t) for t in t_vals]
    img_shape = img[0].shape

# Load MP components
mp_path = '/home/mn2822/Desktop/WormOT/data/zimmer/mp_components/mp_0000_0050.mat'
mp_data = loadmat(mp_path)
cov = mp_data['cov']
pts = [mp_data['means'][t, :, :] for t in t_vals]
wts = [mp_data['weights'][t, :, 0] for t in t_vals]

# Reconstruct images from MPs
rec = [imagerep.reconstruct_image(pts[i], [cov], wts[i], img_shape) for i in range(n_frames)]

# Indices to plot
k1 = 0
k2 = -1
  
plt.figure(figsize=(10, 10))

plt.subplot(221)
plot_maxproj(img[k1])
plt.title(f'frame: {t_vals[k1]}')
plt.axis('off')

plt.subplot(222)
plot_maxproj(img[k2])
plt.title(f'frame: {t_vals[k2]}')
plt.axis('off');

plt.subplot(223)
plot_maxproj(rec[k1])
plt.title('MP recon')
plt.axis('off')

plt.subplot(224)
plot_maxproj(rec[k2])
plt.title('MP recon')
plt.axis('off');

## Approach 1: Register all frames to first frame directly

In [ ]:
models_1 = [None] * (n_frames - 1)
debug_1 = [None] * (n_frames - 1)

for t in range(n_frames - 1):
    models_1[t], debug_1[t] = ot_reg_poly(pts[0], pts[t + 1], wts[0], wts[t + 1], 50)

## Reconstruct frame from mapping

In [ ]:
t = 19

t_rec_pts = models_1[t-1].predict(pts[0])
t_rec = imagerep.reconstruct_image_2(t_rec_pts, wts[0], cov, img_shape)

plt.figure(figsize=(15, 15))

plt.subplot(121)
plot_maxproj(rec[t])
plt.title(f'frame: {t}')
plt.axis('off')
        
plt.subplot(122)
plot_maxproj(t_rec)
plt.title('reconstruction')
plt.axis('off')

## Approach 2: Register all frames to first frame using pushforward

TODO: Re-visit this. Might be a bug -- it should be working at least as well as first approach.

In [ ]:
#models_2 = [None] * (n_frames - 1)
#debug_2 = [None] * (n_frames - 1)

#pf = [None] * n_frames
#pf[0] = pts[0]

#for t in range(n_frames - 1):
    
#    models_2[t], debug_2[t] = ot_reg_poly(pts[0], pts[t + 1], wts[0], wts[t + 1], 50, pf[t])
#    pf[t + 1] = models_2[t].predict(pts[0])

## Reconstruct frame from mapping

In [ ]:
#t = 19

#t_rec_pts = models_2[t-1].predict(pts[0])
#t_rec = imagerep.reconstruct_image_2(t_rec_pts, wts[0], cov, img_shape)

#plt.figure(figsize=(15, 15))

#plt.subplot(121)
#plot_maxproj(rec[t])
#plt.title(f'frame: {t}')
#plt.axis('off')
        
#plt.subplot(122)
#plot_maxproj(t_rec)
#plt.title('reconstruction')
#plt.axis('off')

#plt.savefig('/home/mn2822/Desktop/frame_0_19_recon.png')